In [0]:

df_full = spark.read.format("csv").option("header", "true").load("/Volumes/raw/sys_cadastro/full_load")
df_full.coalesce(1).write.format("delta").saveAsTable("bronze.sys_cadastro.cadastro")
#df_full.display()

In [0]:
%sql
SELECT * FROM bronze.sys_cadastro.cadastro

In [0]:
df_cdc = spark.read.format("csv").option("header", "true").load("/Volumes/raw/sys_cadastro/cdc")
df_cdc.display()

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number,col

#df_cdc = spark.read.format("csv").option("header", "true").load("/Volumes/raw/sys_cadastro/cdc")
#df_full.coalesce(1).write.format("delta").saveAsTable("bronze.sys_cadastro.cadastro")
#df_full.display()
#df_cdc.createOrReplaceGlobalTempView("cadastro")
window_spec = Window.partitionBy("Nome").orderBy(col("modified_date").desc())
df_cdc_unique = df_cdc.withColumn("row_num", row_number().over(window_spec)) \
                .filter("row_num = 1") \
                .drop("row_num")
df_cdc_unique.display()


In [0]:
import delta

bronze = delta.DeltaTable.forName(spark, "bronze.sys_cadastro.cadastro")
bronze

In [0]:
(bronze.alias("b")
    .merge(df_cdc_unique.alias("d"), "b.Nome = d.Nome")
    .whenMatchedDelete(condition= "d.OP = 'D'")
    .whenMatchedUpdateAll(condition= "d.OP = 'U'")
    .whenNotMatchedInsertAll(condition= "d.OP = 'I' OR d.OP = 'U'")
    .execute()
    )
    

In [0]:
%sql
SELECT * FROM bronze.sys_cadastro.cadastro